# Lotto 1등을 향하여!

---
### 1. 데이터 가공

#### 1) 이전 당첨 번호 데이터화

In [1]:
# 라이브러리 불러오기
import random as rd
import pandas as pd
import numpy as np


# 이전 당첨번호 불러오기
bf_win_num = pd.read_csv("last_win_num.csv", index_col="#")
win_num_arr = np.array(bf_win_num)

# 가장 최근 당첨 번호
last_win = list(win_num_arr[0])


#### 2) 이전 당첨 번호 출현 횟수

In [2]:
# 빈도수 카운트 데이터 프레임 제작
wnc = pd.DataFrame()

# 나온 숫자 및 횟수 저장
for n in range(1,7):
    ind = bf_win_num[str(n)].value_counts().index
    va = bf_win_num[str(n)].value_counts().values
    for i in range(ind.shape[0]):
        wnc.loc[ind[i], n] = va[i]
        
# 행을 번호 순서대로 재정렬
ind_wnc = list(wnc.index.values)
ind_wnc_sor = sorted(ind_wnc)
wnc = wnc.reindex(index=ind_wnc_sor)

# 결측치 0으로 변환
wnc.fillna(0, inplace=True)

# 각 번호 별 합, 빈도수 가져오기
wnc["sum"] = wnc[1]+wnc[2]+wnc[3]+wnc[4]+wnc[5]+wnc[6]
wnc["mean_1"] = wnc["sum"] / (wnc["sum"].sum())   # 회차 별 번호 출현 횟수 / 총 추첨 횟수
wnc["mean_t"] = wnc["sum"] / (wnc["sum"].sum()/6) # 회차 별 번호 출현 횟수 / 총 회차 횟수

# 확인
# wnc  # 필요할 때만 주석 제거 후 실행

---
### 2. 뽑기 방법 만들기

#### 1) 무작위 숫자

In [3]:
# 당첨 번호 시뮬레이션 함수 제작
def get_win():
    win = []
    while len(win) <= 5 :
        while True:
            result = rd.randint(1,45)
#             print(result)
            if (result not in last_win) & (result not in win):
                break
        win.append(result)
    win.sort()
    return win

#### 2) 자주 나오던 숫자 자주 나오게 하기(도합 1 가중치)

In [4]:
# 많이 나온 숫자를 많이 자주 나오게 하기
def win_pre_1():
    num_pre = np.random.choice(ind_wnc_sor, size=6, 
                               p=list(wnc["mean_1"]), replace=False)
    num_pre.sort()
    result = list(pd.DataFrame(num_pre)[0])
    return result

#### 3) 자주 나오던 숫자 자주 나오게 하기(상대 가중치)

In [5]:
# 많이 나온 숫자를 많이 나오게 하기 2
def win_pre_t():
    while True :
        pre = rd.choices(ind_wnc_sor, k=6,weights=list(wnc["mean_t"]))
        pre.sort()
        temp = set(pre)
        if len(temp) == 6 :
            break
    result = list(pd.DataFrame(pre)[0])
    return result

#### 4) 아직 많이 안나온 숫자 자주 나오게 하기

---
### 3. 시뮬레이션 옵션 만들기


#### 1) 이전 당첨 번호 조합 겹치지 않게

In [6]:
# 당첨 번호 시뮬레이션
def go_lotto(how) :
    while True:
        win = how
        t_go = 0
        for w in range(len(win_num_arr)) :
            if list(win_num_arr[w]) == win :
                break
            t_go += 1
        if t_go == len(win_num_arr):
            break
    return win

#### 2) 직전 회차 번호 겹치지 않게

In [7]:
def no_last(how):
    while True:
        win = how
        t_no = 0
        for w in last_win :
            if w in win:
                t_no += 1
        if t_no == 0:
            break
    return win

---
### 4. 시뮬레이션 돌리기

- 방법 : 시뮬레이션 옵션( 추첨방법( ) )

#### 1) 시뮬레이션 옵션 정리

- go_lotto() : 이전 당첨 조합과 미일치 옵션

#### 1) 시뮬레이션 옵션 정리

- get_win() : 무작위 추첨(ver.직전 회차 번호 미포함)
- win_pre_1() : 자주 나온 번호 자주 나오게 하기(ver.확률 합 1)
- win_pre_t() : 자주 나온 버놓 자주 나오게 하기(ver.상대 출현률)

In [8]:
# 연결 완료
rd.randint(1,1000)

304

In [ ]:
no_last(win_pre_1())